In [16]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import calendar
from datetime import datetime, timedelta

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, Add, GlobalAveragePooling1D
from tensorflow.python.client import device_lib
from keras.models import load_model

from sklearn.metrics import r2_score
from arch import arch_model
from scipy import stats as sts
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
# List all physical devices TensorFlow can see
print("All physical devices:", tf.config.list_physical_devices())
print("All devices: ",device_lib.list_local_devices())
tf.keras.mixed_precision.set_global_policy('mixed_float16')
# Specifically list GPU devices
print("GPU devices:", tf.config.list_physical_devices('GPU'))

# Optional: Turn on device placement logging
tf.debugging.set_log_device_placement(False)

# Run a simple test
a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
b = tf.matmul(a, a)
print("Result of matmul:", b)

All physical devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
All devices:  [name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16389173399254034979
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
locality {
  bus_id: 1
}
incarnation: 3922184299274304654
physical_device_desc: "device: 0, name: METAL, pci bus id: <undefined>"
xla_global_id: -1
]
GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Result of matmul: tf.Tensor(
[[ 7. 10.]
 [15. 22.]], shape=(2, 2), dtype=float32)


2025-07-29 20:46:07.965699: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-07-29 20:46:07.965728: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [18]:
file = '/Users/damensavvasavvi/Desktop/ML/AlgoTrading/marketDataNasdaqFutures/1m.csv'
df = pd.read_csv(file,usecols=['open','high','low','close','volume','ts_event'], parse_dates=['ts_event'])
time = df['ts_event'].dt
df = df[((time.year<2024) & (time.month<7)) & (time.year>2019)] # Saving last 12 months of data for walk forward testing
# Excluding weekends and non-trading hours
df = df[
    (df['ts_event'].dt.dayofweek < 5) &  # Mon-Fri
    (df['ts_event'].dt.time >= pd.to_datetime('07:00').time()) &  # London open
    (df['ts_event'].dt.time <= pd.to_datetime('21:00').time())  # NY close
]

In [19]:
# Remove outliers in the 'close' column using Z-score and percentage change
z_score_price = np.abs(sts.zscore(df['close']))
percentage_change_price = df['close'].pct_change().fillna(0)
df['close'] = df['close'].where((z_score_price < 3) & (percentage_change_price<0.5), np.nan).interpolate(method='linear')

z_score_volume = np.abs(sts.zscore(df['volume']))
df['volume'] = df['volume'].where(z_score_volume < 3, np.nan).interpolate(method='polynomial', order=5)

In [20]:
# 5 minute data
dataClose = df['close']
dataClose = dataClose.replace(0, np.nan).ffill()

time = df['ts_event']

time_since_start_year = (time - time.min()).dt.days
time_since_start_year_normalised = (time_since_start_year - time_since_start_year.mean())/ time_since_start_year.std()

months_sin = np.sin(2*np.pi * time.dt.month/12)
months_cos = np.cos(2*np.pi * time.dt.month/12)

day_of_week_sin = np.sin(2*np.pi * time.dt.dayofweek/7)
day_of_week_cos = np.cos(2*np.pi * time.dt.dayofweek/7)

hours_sin = np.sin(2*np.pi*time.dt.hour/24)
hours_cos = np.cos(2*np.pi*time.dt.hour/24)

minutes_sin = np.sin(2*np.pi*time.dt.minute/60)
minutes_cos = np.cos(2*np.pi*time.dt.minute/60)

time_features = pd.DataFrame({
    'time_since_start_year_normalised': time_since_start_year_normalised,
    'months_sin': months_sin,
    'months_cos': months_cos,
    'day_of_week_sin': day_of_week_sin,
    'day_of_week_cos': day_of_week_cos,
    'hours_sin': hours_sin,
    'hours_cos': hours_cos,
    'minutes_sin': minutes_sin,
    'minutes_cos': minutes_cos
})
time_features
dataClose

230702     12893.75
230703     12894.00
230704     12893.25
230705     12900.25
230706     12899.00
             ...   
1508638    15513.75
1508639    15316.50
1508640    15317.00
1508641    15316.50
1508642    15316.50
Name: close, Length: 502175, dtype: float64

In [21]:
def calculate_bollinger_bands(data, window=14, num_of_std=2):
    """Calculate Bollinger Bands"""
    rolling_mean = data.rolling(window=window).mean()
    rolling_std = data.rolling(window=window).std()
    upper_band = rolling_mean + (rolling_std * num_of_std)
    lower_band = rolling_mean - (rolling_std * num_of_std)
    return upper_band, lower_band

def calculate_rsi(data, window=10):
    """Calculate Relative Strength Index"""
    delta = data.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_roc(data, periods=10):
    """Calculate Rate of Change."""
    roc = ((data - data.shift(periods)) / data.shift(periods)) * 100
    return roc

In [22]:
def calculate_vwap(df):
    """
    Calculate the Volume Weighted Average Price (VWAP).
    VWAP = sum(price * volume) / sum(volume)
    Assumes df has columns: 'close', 'volume'
    Returns a pandas Series with VWAP values.
    """
    vwap = (df['close'] * df['volume']).cumsum() / df['volume'].cumsum()
    return vwap

def calculate_atr(df, window=14):
    """
    Calculate the Average True Range (ATR).
    ATR is a measure of volatility.
    Assumes df has columns: 'high', 'low', 'close'
    Returns a pandas Series with ATR values.
    """
    high_low = df['high'] - df['low']
    high_close = (df['high'] - df['close'].shift(1)).abs()
    low_close = (df['low'] - df['close'].shift(1)).abs()
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    atr = true_range.rolling(window=window, min_periods=1).mean()
    return atr

In [23]:
extension = [dataClose.index[-1] + i for i in range(1, 31)]
dataClose_extended = pd.Series(np.concatenate([dataClose.values, np.full(30, np.nan)]), index=dataClose.index.append(pd.Index(extension)))

In [24]:
def garch(dataClose, forecast_horizon=30):
    """Fit a GARCH model to the log returns of the closing prices. Optimises the model by selecting the one with the lowest AIC."""
    from arch import arch_model
    # Calculate log returns
    returns = np.log(dataClose / dataClose.shift(1)).interpolate(method='linear').bfill()

    best_aic = float('inf')
    best_model = None

    for p in range(1, 3):
        for q in range(1, 3):
            model = arch_model(returns, vol='GARCH', p=p, q=q).fit(disp='off')
            if model.aic < best_aic:
                best_aic = model.aic
                best_model = model
    # Fit the best model
    volatility = best_model.conditional_volatility
    # Align and create a new feature column
    vol_feature = volatility.shift(1).bfill()

    # Forecast conditional volatility h steps ahead
    forecast = best_model.forecast(horizon=forecast_horizon)
    
    # Get forecasted conditional variances
    var_forecast = forecast.variance.values[-1, :]  # shape (horizon,)
    
    # Convert variance to std dev (volatility)
    vol_forecast = np.sqrt(var_forecast)
    return vol_feature, vol_forecast
vol_feature, vol_forecast = garch(dataClose)

# Assuming vol_feature is a Series and vol_forecast is a numpy array
combined_vol = pd.Series(np.concatenate([vol_feature.values, vol_forecast]), index=dataClose_extended.index)

# Shift the combined series if you want to align it for prediction, e.g., shift by -5
combined_vol = combined_vol.shift(-30).dropna()
combined_vol.tail(3000)


/opt/miniconda3/envs/tf_py3.12.6/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/miniconda3/envs/tf_py3.12.6/lib/python3.12/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/opt/miniconda3/envs/tf_py3.12.6/lib/python3.12/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


1504497    1.236362
1504498    1.163841
1504499    1.098042
1504500    2.238434
1504501    2.329842
             ...   
1508638    1.234406
1508639    1.252945
1508640    1.271226
1508641    1.289260
1508642    1.307057
Length: 3000, dtype: float64

In [25]:

from statsmodels.tsa.arima.model import ARIMA

def arima_forecast(dataClose, forecast_horizon=30,order=(3,1,3),gridsearch=True, max_p=3, max_d=2, max_q=3):
    """
    Fit ARIMA(p,d,q) models on log returns, select best by AIC, and forecast returns.
    
    Parameters:
    - dataClose: pd.Series of closing prices
    - forecast_horizon: number of steps to forecast ahead
    - max_p, max_d, max_q: maximum orders to search for
    
    Returns:
    - fitted_returns: pd.Series of in-sample fitted values (returns)
    - forecast_returns: np.ndarray of forecasted returns for horizon
    - best_order: tuple (p,d,q) of best ARIMA order
    """
    # Calculate log returns
    returns = np.log(dataClose / dataClose.shift(1)).interpolate(method='linear').bfill()

    best_aic = float('inf')
    best_order = None
    best_model = None
    
    if gridsearch:
        # Grid search over p,d,q
        for p in range(max_p+1):
            for d in range(max_d+1):
                for q in range(max_q+1):
                    if p == 0 and d == 0 and q == 0:
                        continue  # skip trivial model
                    try:
                        model = ARIMA(returns, order=(p,d,q))
                        fitted = model.fit()
                        if fitted.aic < best_aic:
                            best_aic = fitted.aic
                            best_order = (p,d,q)
                            best_model = fitted
                    except:
                        # Some parameter combos may fail to converge; skip those
                        continue
    else:
        # Use the provided order directly
        best_model = ARIMA(returns, order=order).fit()
        
    # Get in-sample fitted values (returns)
    fitted_returns = pd.Series(best_model.fittedvalues, index=returns.index)
    
    # Forecast returns for forecast_horizon steps ahead
    forecast = best_model.get_forecast(steps=forecast_horizon)
    forecast_returns = forecast.predicted_mean.values
    
    return fitted_returns, forecast_returns, best_order

# Example usage:
fitted_returns, forecast_returns, best_order = arima_forecast(dataClose, gridsearch=False)

# Best ARIMA order: (3, 1, 3)
# Next 5 forecasted returns: [-0.16682214  0.04469367 -0.05550041  0.05141629 -0.05280984]


/opt/miniconda3/envs/tf_py3.12.6/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/miniconda3/envs/tf_py3.12.6/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/opt/miniconda3/envs/tf_py3.12.6/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/opt/miniconda3/envs/tf_py3.12.6/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use

In [26]:

combined_arima = pd.Series(np.concatenate([fitted_returns.values, forecast_returns]), index=dataClose_extended.index).shift(-30).dropna()
combined_arima.tail(3000)

1504497   -0.151231
1504498   -0.124564
1504499    2.838296
1504500    0.582780
1504501   -0.536480
             ...   
1508638   -0.005997
1508639    0.006103
1508640   -0.005987
1508641    0.006093
1508642   -0.005977
Length: 3000, dtype: float64

In [27]:
# Calulate rolling linear regression slopes across a window of 9
window = 9
slopes = []

for i in range(window - 1, len(dataClose)):
    window_data = dataClose.iloc[i - window + 1 : i + 1]

    # Handle missing data (interpolate + fill)
    if window_data.isna().any():
        window_data = window_data.interpolate().bfill().ffill()

    y = window_data.values.reshape(-1, 1)  # shape (9,1)
    X = np.arange(window).reshape(-1, 1)  # shape (9,1)

    model = LinearRegression().fit(X, y)
    slopes.append(model.coef_[0][0])


# Convert to pandas Series, aligned with dataClose
slopes_series = pd.Series([np.nan]*(window-1) + slopes, index=dataClose.index).interpolate(method='linear', limit_direction='both')



In [28]:
upper_band, lower_band = calculate_bollinger_bands(dataClose)
rsi = calculate_rsi(dataClose).bfill()
rsi_rolling = rsi.rolling(window=14).mean().bfill()  # Rolling mean of RSI
roc = calculate_roc(dataClose).bfill()
bb_width = upper_band.bfill() - lower_band.bfill()
diff = dataClose.diff().fillna(0)
percent_change = dataClose.pct_change().bfill() * 100
vwap = calculate_vwap(df).bfill()  # Calculate VWAP for 5-minute data
atr = calculate_atr(df).bfill()  # Calculate ATR for 5-minute data
volume = df['volume'].replace(0, np.nan).bfill()  # Volume data for 5-minute intervals
ma_9 = dataClose.rolling(window=9).mean().interpolate(method='linear', limit_direction='both') # 9-period moving average

features_dict = {
    'dataClose': dataClose,  # Closing price
    'bb_width': bb_width,   # Bollinger Bands Width
    'rsi': rsi,             # Relative Strength Index
    'rsi_rolling': rsi_rolling,  # Rolling mean of RSI
    'roc': roc,             # Rate of Change
    'diff': diff,           # Price Difference
    'percent_change': percent_change,  # Percentage Change
    'vol_feature': combined_vol,
    'dataClose_lag1': dataClose.shift(1).bfill(),  # Lagged closing price
    'dataClose_lag3': dataClose.shift(3).bfill(),  # Lagged closing price (3 periods)
    'dataClose_lag6': dataClose.shift(6).bfill(),  # Lagged closing price (6 periods)
    'bb_width_lag1': bb_width.shift(1).bfill(),  # Lagged Bollinger Bands Width
    'bb_width_lag3': bb_width.shift(3).bfill(),  # Lagged Bollinger Bands Width (3 periods)
    'bb_width_lag6': bb_width.shift(6).bfill(),  # Lagged Bollinger Bands Width (6 periods)
    'rsi_lag1': rsi.shift(1).bfill(),  # Lagged RSI
    'rsi_lag3': rsi.shift(3).bfill(),  # Lagged RSI (3 periods)
    'rsi_lag6': rsi.shift(6).bfill(),  # Lagged RSI (6 periods)
    'roc_lag1': roc.shift(1).bfill(),  # Lagged ROC
    'roc_lag3': roc.shift(3).bfill(),  # Lagged ROC (3 periods)
    'roc_lag6': roc.shift(6).bfill(),  # Lagged ROC (6 periods)
    'diff_lag1': diff.shift(1).bfill(),  # Lagged Price Difference
    'diff_lag3': diff.shift(3).bfill(),  # Lagged Price Difference (3 periods)
    'diff_lag6': diff.shift(6).bfill(),  # Lagged Price Difference (6 periods)
    'percent_change_lag1': percent_change.shift(1).bfill(),  # Lagged Percentage Change
    'percent_change_lag3': percent_change.shift(3).bfill(),  # Lagged Percentage Change (3 periods)
    'percent_change_lag6': percent_change.shift(6).bfill(),  # Lagged Percentage Change (6 periods)
    'vol_feature_lag1': vol_feature.shift(1).bfill(),  # Lagged Volatility Feature
    'vol_feature_lag3': vol_feature.shift(3).bfill(),  # Lagged Volatility Feature (3 periods)
    'vol_feature_lag6': vol_feature.shift(6).bfill(),  # Lagged Volatility Feature (6 periods)
    'vwap': vwap,  # Volume Weighted Average Price
    'atr': atr,  # Average True Range
    'volume': volume,  # Volume data for 5-minute intervals
    'vwap_lag1': vwap.shift(1).bfill(),  # Lagged VWAP
    'vwap_lag3': vwap.shift(3).bfill(),  # Lagged VWAP (3 periods)
    'vwap_lag6': vwap.shift(6).bfill(),  # Lagged VWAP (6 periods)
    'atr_lag1': atr.shift(1).bfill(),  # Lagged ATR
    'atr_lag3': atr.shift(3).bfill(),  # Lagged ATR (3 periods)
    'atr_lag6': atr.shift(6).bfill(),    # Lagged ATR (6 periods)
    'volume_lag1': volume.shift(1).bfill(),  # Lagged Volume
    'volume_lag3': volume.shift(3).bfill(),  # Lagged   Volume (3 periods)
    'volume_lag6': volume.shift(6).bfill(),  # Lagged Volume (6 periods)
    'combined_arima': combined_arima,  # Combined ARIMA forecast
    'combined_vol': combined_vol,  # Combined GARCH forecast
    'ma_9': ma_9,  # 9-period moving average
    'linear_regression_slope': slopes_series,  # Linear regression slope

}
features = pd.DataFrame(features_dict)

# {mean,std for each feature}
stats = {
         "bb_width": (bb_width.mean(), bb_width.std()),
         "rsi": (rsi.mean(), rsi.std()),
         'rsi_rolling': (rsi_rolling.mean(), rsi_rolling.std()),
         "roc": (roc.mean(), roc.std()),
         "diff": (diff.mean(), diff.std()),
         "percent_change": (percent_change.mean(), percent_change.std()),
         "dataClose": (dataClose.mean(), dataClose.std()),
         "vol_feature": (vol_feature.mean(), vol_feature.std()),
         "dataClose_lag1": (dataClose.shift(1).mean(), dataClose.shift(1).std()),
         "dataClose_lag3": (dataClose.shift(3).mean(), dataClose.shift(3).std()),
         "dataClose_lag6": (dataClose.shift(6).mean(), dataClose.shift(6).std()),
            "bb_width_lag1": (bb_width.shift(1).mean(), bb_width.shift(1).std()),
            "bb_width_lag3": (bb_width.shift(3).mean(), bb_width.shift(3).std()),
            "bb_width_lag6": (bb_width.shift(6).mean(), bb_width.shift(6).std()),
            "rsi_lag1": (rsi.shift(1).mean(), rsi.shift(1).std()),
            "rsi_lag3": (rsi.shift(3).mean(), rsi.shift(3).std()),
            "rsi_lag6": (rsi.shift(6).mean(), rsi.shift(6).std()),
            "roc_lag1": (roc.shift(1).mean(), roc.shift(1).std()),
            "roc_lag3": (roc.shift(3).mean(), roc.shift(3).std()),
            "roc_lag6": (roc.shift(6).mean(), roc.shift(6).std()),
            "diff_lag1": (diff.shift(1).mean(), diff.shift(1).std()),
            "diff_lag3": (diff.shift(3).mean(), diff.shift(3).std()),
            "diff_lag6": (diff.shift(6).mean(), diff.shift(6).std()),
            "percent_change_lag1": (percent_change.shift(1).mean(), percent_change.shift(1).std()),
            "percent_change_lag3": (percent_change.shift(3).mean(), percent_change.shift(3).std()),
            "percent_change_lag6": (percent_change.shift(6).mean(), percent_change.shift(6).std()),
            "vol_feature_lag1": (vol_feature.shift(1).mean(), vol_feature.shift(1).std()),
            "vol_feature_lag3": (vol_feature.shift(3).mean(), vol_feature.shift(3).std()),
            "vol_feature_lag6": (vol_feature.shift(6).mean(), vol_feature.shift(6).std()),
            "vwap": (vwap.mean(), vwap.std()),
            "atr": (atr.mean(), atr.std()),
            "vwap_lag1": (vwap.shift(1).mean(), vwap.shift(1).std()),
            "vwap_lag3": (vwap.shift(3).mean(), vwap.shift(3).std()),
            "vwap_lag6": (vwap.shift(6).mean(), vwap.shift(6).std()),
            "atr_lag1": (atr.shift(1).mean(), atr.shift(1).std()),
            "atr_lag3": (atr.shift(3).mean(), atr.shift(3).std()),
            "atr_lag6": (atr.shift(6).mean(), atr.shift(6).std()),
            "volume": (volume.mean(), volume.std()),
            "volume_lag1": (volume.shift(1).mean(), volume.shift(1).std()),
            "volume_lag3": (volume.shift(3).mean(), volume.shift(3).std()),
            "volume_lag6": (volume.shift(6).mean(), volume.shift(6).std()),
            "combined_arima": (combined_arima.mean(), combined_arima.std()),
            "combined_vol": (combined_vol.mean(), combined_vol.std()),
            "ma_9": (ma_9.mean(), ma_9.std()),
            'linear_regression_slope': (slopes_series.mean(), slopes_series.std()),
            
         }
features


,dataClose,bb_width,rsi,rsi_rolling,roc,diff,percent_change,vol_feature,dataClose_lag1,dataClose_lag3,...,atr_lag1,atr_lag3,atr_lag6,volume_lag1,volume_lag3,volume_lag6,combined_arima,combined_vol,ma_9,linear_regression_slope
230702,12893.75,28.566195,100.000000,66.939687,0.104702,0.00,0.001939,0.549345,12893.75,12893.75,...,6.000000,6.000000,6.000000,338.0,338.0,338.0,0.001102,0.549345,12896.472222,0.566667
230703,12894.00,28.566195,100.000000,66.939687,0.104702,0.25,0.001939,0.549727,12893.75,12893.75,...,6.000000,6.000000,6.000000,338.0,338.0,338.0,0.000938,0.549727,12896.472222,0.566667
230704,12893.25,28.566195,25.000000,66.939687,0.104702,-0.75,-0.005817,0.550054,12894.00,12893.75,...,3.750000,6.000000,6.000000,95.0,338.0,338.0,0.000853,0.550054,12896.472222,0.566667
230705,12900.25,28.566195,90.625000,66.939687,0.104702,7.00,0.054292,0.550333,12893.25,12893.75,...,3.250000,6.000000,6.000000,115.0,338.0,338.0,0.000746,0.550333,12896.472222,0.566667
230706,12899.00,28.566195,78.378378,66.939687,0.104702,-1.25,-0.009690,0.550574,12900.25,12894.00,...,4.812500,3.750000,6.000000,538.0,95.0,338.0,0.000645,0.550574,12896.472222,0.566667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1508638,15513.75,282.694415,98.886139,53.846832,1.289480,195.75,1.277908,1.234406,15318.00,15317.50,...,29.089286,29.107143,29.303571,122.0,109.0,67.0,-0.005997,1.234406,15338.500000,13.383333
1508639,15316.50,282.476032,49.968652,54.054677,-0.001632,-197.25,-1.271453,1.252945,15513.75,15317.25,...,42.857143,29.089286,29.267857,1.0,30.0,29.0,0.006103,1.252945,15338.611111,10.000000
1508640,15317.00,282.415883,50.188442,53.461668,0.009794,0.50,0.003264,1.271226,15316.50,15318.00,...,56.875000,29.089286,29.178571,144.0,122.0,20.0,-0.005987,1.271226,15338.777778,6.633333
1508641,15316.50,210.785136,50.125471,50.091779,0.006529,-0.50,-0.003264,1.289260,15317.00,15513.75,...,56.785714,42.857143,29.107143,30.0,1.0,109.0,0.006093,1.289260,15338.750000,3.312500


In [29]:
# In-place standardization
def normalize_data(feature):
    mean = features[feature].mean()
    std = features[feature].std()
    features[feature] = (features[feature] - mean) / std
    print(f"Normalized {feature}: mean={mean}, std={std}")

def remove_outliers(series, threshold=3):
    """Remove outliers based on Z-score."""
    z_scores = np.abs(sts.zscore(series))
    return series.where(z_scores < threshold, np.nan).interpolate(method='linear')

# Loop through features one by one
for feature in features.columns:
    features[feature] = remove_outliers(features[feature])
    # Normalize feature in-place
    normalize_data(feature)

# Save to CSV
features = pd.concat([time_features, features], axis=1)
features.to_csv('features.csv', index=True)
stats = pd.DataFrame(stats)
stats.to_csv('feature_stats.csv', index=True)
stats

Normalized dataClose: mean=11091.96548728033, std=4893.0388930905865
Normalized bb_width: mean=14252.126005957742, std=10212.42172693081
Normalized rsi: mean=50.131094565527974, std=10.980131920758037
Normalized rsi_rolling: mean=50.53996813771397, std=5.661662010546978
Normalized roc: mean=-7970.11814588502, std=87125.07936428429
Normalized diff: mean=0.004824513366852498, std=5951.443299811186
Normalized percent_change: mean=-11317.339944902747, std=82937.35324751082
Normalized vol_feature: mean=1.4060045412952509, std=1.1654422638801074
Normalized dataClose_lag1: mean=11091.96066276696, std=4893.035922120262
Normalized dataClose_lag3: mean=11091.951012744561, std=4893.029979300239
Normalized dataClose_lag6: mean=11091.936143426097, std=4893.020716682841
Normalized bb_width_lag1: mean=14252.125643139905, std=10212.422229017291
Normalized bb_width_lag3: mean=14252.124774780012, std=10212.42342892571
Normalized bb_width_lag6: mean=14252.123411398916, std=10212.425312182451
Normalized r

,bb_width,rsi,rsi_rolling,roc,diff,percent_change,dataClose,vol_feature,dataClose_lag1,dataClose_lag3,...,atr_lag3,atr_lag6,volume,volume_lag1,volume_lag3,volume_lag6,combined_arima,combined_vol,ma_9,linear_regression_slope
0,14252.126006,50.535064,50.535208,-10559.915274,0.004825,-19969.958485,11091.965487,1.421239,11091.957075,11091.940249,...,3521.910867,3521.931650,366.101265,366.101717,366.103006,366.104661,-0.001107,1.421270,11091.946042,0.004895
1,10212.421727,15.429803,6.049559,426825.437003,5951.443300,358308.038816,4893.038893,1.215875,4893.040133,4893.042613,...,3135.493982,3135.491817,550.613527,550.613982,550.614699,550.615906,1.048672,1.215859,2611.349430,560.585726


In [ ]:
# Moving all data points forward to prevent data leakage
labels = features.shift(-1)
labels = labels.iloc[:-1]

features = features.iloc[:-1]  # Align features with labels
labels


,time_since_start_year_normalised,months_sin,months_cos,day_of_week_sin,day_of_week_cos,hours_sin,hours_cos,minutes_sin,minutes_cos,dataClose,...,atr_lag1,atr_lag3,atr_lag6,volume_lag1,volume_lag3,volume_lag6,combined_arima,combined_vol,ma_9,linear_regression_slope
230702,-1.530026,5.000000e-01,0.866025,0.000000,1.000000,0.965926,-0.258819,0.104528,0.994522,0.368285,...,-1.121321,-1.121321,-1.121321,0.013566,0.013563,0.013560,0.106772,-0.734723,0.692251,0.002913
230703,-1.530026,5.000000e-01,0.866025,0.000000,1.000000,0.965926,-0.258819,0.207912,0.978148,0.368132,...,-1.122039,-1.121321,-1.121321,-0.513356,0.013563,0.013560,0.106668,-0.734443,0.692251,0.002913
230704,-1.530026,5.000000e-01,0.866025,0.000000,1.000000,0.965926,-0.258819,0.309017,0.951057,0.369563,...,-1.122198,-1.121321,-1.121321,-0.469988,0.013563,0.013560,0.106537,-0.734203,0.692251,0.002913
230705,-1.530026,5.000000e-01,0.866025,0.000000,1.000000,0.965926,-0.258819,0.406737,0.913545,0.369307,...,-1.121700,-1.122039,-1.121321,0.447246,-0.513359,0.013560,0.106413,-0.733997,0.692251,0.002913
230706,-1.530026,5.000000e-01,0.866025,0.000000,1.000000,0.965926,-0.258819,0.500000,0.866025,0.369512,...,-1.121784,-1.122198,-1.121321,-0.179422,-0.469990,0.013560,0.106338,-0.733820,0.692251,0.002913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1508637,1.506589,1.224647e-16,-1.000000,-0.433884,-0.900969,-0.866025,0.500000,-0.500000,0.866025,0.903689,...,-1.113957,-1.113952,-1.113889,-0.454809,-0.483001,-0.574078,0.098269,-0.147239,1.630570,0.026670
1508638,1.506589,1.224647e-16,-1.000000,-0.433884,-0.900969,-0.866025,0.500000,-0.406737,0.913545,0.863376,...,-1.109566,-1.113957,-1.113900,-0.717185,-0.654305,-0.656477,0.113105,-0.131332,1.630612,0.020398
1508639,1.506589,1.224647e-16,-1.000000,-0.433884,-0.900969,-0.866025,0.500000,-0.309017,0.951057,0.863479,...,-1.105096,-1.113957,-1.113929,-0.407104,-0.454812,-0.675993,0.098281,-0.115646,1.630676,0.014158
1508640,1.506589,1.224647e-16,-1.000000,-0.433884,-0.900969,-0.866025,0.500000,-0.207912,0.978148,0.863376,...,-1.105124,-1.109566,-1.113951,-0.654302,-0.717188,-0.483005,0.113093,-0.100172,1.630666,0.008002


In [12]:
# This paramater is up for optimization, but 24 is a good starting point
SEQUENCE_LEN = 24  # 2 hours of data at 5-minute intervals

def create_sequences(data, labels, mean, std, sequence_length=SEQUENCE_LEN):
    sequences = []
    lab = []
    data_size = len(data)

    # Loop to create each sequence and its corresponding label
    for i in range(data_size - (sequence_length + 13)): # Ensure we have data for the label
        if i == 0:
          continue
        sequences.append(data[i:i + sequence_length])  # The sequence of data
        lab.append([labels[i-1], labels[i + 12], mean, std]) # The label and scaling factors

    return np.array(sequences), np.array(lab)

In [13]:
NQ_data = np.column_stack([features[col] for col in features.columns])

data_sequences, data_labels = create_sequences(NQ_data, labels['dataClose'].values[SEQUENCE_LEN-1:], stats['dataClose'][0], stats['dataClose'][1])



In [ ]:
np.random.seed(42)
shuffled_indices = np.random.permutation(len(data_sequences))
data_sequences = data_sequences[shuffled_indices]
data_labels = data_labels[shuffled_indices]

train_size = int(len(data_sequences) * 0.8)

train_sequences = data_sequences[:train_size]
train_labels = data_labels[:train_size]

other_sequences = data_sequences[train_size:]
other_labels = data_labels[train_size:]

shuffled_indices = np.random.permutation(len(other_sequences))
other_sequences = other_sequences[shuffled_indices]
other_labels = other_labels[shuffled_indices]

val_size = int(len(other_sequences) * 0.5)
val_sequences = other_sequences[:val_size]
val_labels = other_labels[:val_size]

test_sequences = other_sequences[val_size:]
test_labels = other_labels[val_size:]
print("test_sequences shape:", test_sequences.shape)






NameError: name 'data_sequences' is not defined

In [15]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and normalization
    x = LayerNormalization(epsilon=1e-6)(inputs)
    x = MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = Add()([x, inputs])

    # Feed Forward Network
    y = LayerNormalization(epsilon=1e-6)(x)
    y = Dense(ff_dim, activation="relu")(y)
    y = Dropout(dropout)(y)
    y = Dense(inputs.shape[-1])(y)
    return Add()([y, x])

In [16]:
def build_transformer_model(input_shape, head_size, num_heads, ff_dim, num_layers, dropout=0):
    inputs = Input(shape=input_shape)
    x = inputs

    for _ in range(num_layers):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)
    x = GlobalAveragePooling1D()(x)
    x = LayerNormalization(epsilon=1e-6)(x)
    outputs = Dense(1, activation="linear")(x)  # Output layer for regression
    return Model(inputs, outputs)

In [17]:
input_shape = train_sequences.shape[1:]
head_size = 256
num_heads = 16
ff_dim = 1024
num_layers = 12
dropout = 0.20
# input_shape = train_sequences.shape[1:]
# head_size = 128   # Reduced from 256
# num_heads = 8     # Reduced from 16
# ff_dim = 512      # Reduced from 1024
# num_layers = 6    # Reduced from 12 (most impactful!)
# dropout = 0.20    # Keep as is, or adjust later if overfitting/underfitting

# --- RECOMMENDED CHANGES (Set B) ---
# input_shape = train_sequences.shape[1:]
# head_size = 64    # Further reduced from 128
# num_heads = 4     # Further reduced from 8
# ff_dim = 256      # Further reduced from 512
# num_layers = 4    # Further reduced from 6
# dropout = 0.20

# --- RECOMMENDED CHANGES (Set C) MEDIUM SIZE---
# head_size = 128     # Stepping up from 64, half of original 256
# num_heads = 8       # Stepping up from 4, half of original 16
# ff_dim = 512        # Stepping up from 256, half of original 1024
# num_layers = 8      # Significant jump from 4 or 6, but not the original 12

# dropout = 0.20      # Keep as is, adjust later if needed for overfitting/underfitting


In [18]:
def custom_mae_loss(y_true, y_pred):
    y_true_next = tf.cast(y_true[:, 1], tf.float64)  # Extract the true next values, scaled
    y_pred_next = tf.cast(y_pred[:, 0], tf.float64)  # Extract the predicted next values, scaled
    abs_error = tf.abs(y_true_next - y_pred_next)  # Calculate the absolute error
    return tf.reduce_mean(abs_error)  # Return the mean of these errors

def dir_acc(y_true, y_pred):
    mean, std = tf.cast(y_true[:, 2], tf.float64), tf.cast(y_true[:, 3], tf.float64)  # Retrieve scaling factors
    y_true_prev = (tf.cast(y_true[:, 0], tf.float64) * std) + mean  # Un-scale previous true price
    y_true_next = (tf.cast(y_true[:, 1], tf.float64) * std) + mean  # Un-scale next true price
    y_pred_next = (tf.cast(y_pred[:, 0], tf.float64) * std) + mean  # Un-scale predicted next price

    true_change = y_true_next - y_true_prev  # Calculate true change
    pred_change = y_pred_next - y_true_prev  # Calculate predicted change

    correct_direction = tf.equal(tf.sign(true_change), tf.sign(pred_change))  # Check if the signs match
    return tf.reduce_mean(tf.cast(correct_direction, tf.float64))  # Return the mean of correct directions



In [19]:

model = build_transformer_model(input_shape, head_size, num_heads, ff_dim, num_layers, dropout)
model.summary()
model.compile(
      loss=custom_mae_loss,  # Custom loss function
      optimizer=tf.keras.optimizers.Adam(),
      metrics=[dir_acc]  # Directional accuracy as a metric
)



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 24, 54)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 24, 54)    │        108 │ input_layer[0][0] │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 24, 54)    │    897,078 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 24, 54)    │          0 │ multi_head_atten… │
│                     │                   │            │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 24, 54)    │        108 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 24, 1024)  │     56,320 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 24, 1024)  │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 24, 54)    │     55,350 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 24, 54)    │          0 │ dense_1[0][0],    │
│                     │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 24, 54)    │        108 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 24, 54)    │    897,078 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 24, 54)    │          0 │ multi_head_atten… │
│                     │                   │            │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 24, 54)    │        108 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 24, 1024)  │     56,320 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 24, 1024)  │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 24, 54)    │     55,350 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 24, 54)    │          0 │ dense_3[0][0],    │
│                     │                   │            │ add_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 24, 54)    │        108 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 24, 54)    │    897,078 │ layer_normalizat

 Total params: 12,107,731 (46.19 MB)

 Trainable params: 12,107,731 (46.19 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# Define a callback to save the best model
checkpoint_callback_train = ModelCheckpoint(
    "transformer_train_model.keras",  # Filepath to save the best model
    monitor="dir_acc",  #"loss",  # Metric to monitor
    save_best_only=True,  # Save only the best model
    mode="max",  # Minimize the monitored metric
    verbose=1,  # Display progress
)

# Define a callback to save the best model
checkpoint_callback_val = ModelCheckpoint(
    "transformer_val_model.keras",  # Filepath to save the best model
    monitor="val_dir_acc", #"val_loss",  # Metric to monitor
    save_best_only=True,  # Save only the best model
    mode="max",  # Minimize the monitored metric
    verbose=1,  # Display progress
)

def get_lr_callback(batch_size=16, mode='cos', epochs=500, plot=False):
    lr_start, lr_max, lr_min = 0.0001, 0.005, 0.00001  # Adjust learning rate boundaries
    lr_ramp_ep = int(0.30 * epochs)  # 30% of epochs for warm-up
    lr_sus_ep = max(0, int(0.10 * epochs) - lr_ramp_ep)  # Optional sustain phase, adjust as needed

    def lrfn(epoch):
        if epoch < lr_ramp_ep:  # Warm-up phase
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
        elif epoch < lr_ramp_ep + lr_sus_ep:  # Sustain phase at max learning rate
            lr = lr_max
        elif mode == 'cos':
            decay_total_epochs, decay_epoch_index = epochs - lr_ramp_ep - lr_sus_ep, epoch - lr_ramp_ep - lr_sus_ep
            phase = math.pi * decay_epoch_index / decay_total_epochs
            lr = (lr_max - lr_min) * 0.5 * (1 + math.cos(phase)) + lr_min
        else:
            lr = lr_min  # Default to minimum learning rate if mode is not recognized

        return lr

    if plot:  # Plot learning rate curve if plot is True
        plt.figure(figsize=(10, 5))
        plt.plot(np.arange(epochs), [lrfn(epoch) for epoch in np.arange(epochs)], marker='o')
        plt.xlabel('Epoch')
        plt.ylabel('Learning Rate')
        plt.title('Learning Rate Scheduler')
        plt.show()

    return tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)

In [ ]:
BATCH_SIZE = 128  # Number of training examples used to calculate each iteration's gradient
EPOCHS = 150  # Total number of times the entire dataset is passed through the network

import os



train_dataset = tf.data.Dataset.from_tensor_slices((train_sequences, train_labels))
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = tf.data.Dataset.from_tensor_slices((val_sequences, val_labels))
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

model.fit(
    train_sequences,  # Training features
    train_labels,  # Training labels
    validation_data=(val_sequences, val_labels),  # Validation data
    epochs=EPOCHS,  # Number of epochs to train for
    batch_size=BATCH_SIZE,  # Size of each batch
    initial_epoch=model.optimizer.iterations.numpy() // (len(train_sequences) // BATCH_SIZE),  # Start from the last saved epoch
    shuffle=True,  # Shuffle training data before each epoch
    callbacks=[checkpoint_callback_train, checkpoint_callback_val, get_lr_callback(batch_size=BATCH_SIZE, epochs=EPOCHS)]  # Callbacks for saving models and adjusting learning rate
)

2025-07-29 14:49:10.972400: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-07-29 14:49:10.972414: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2025-07-29 14:49:10.972709: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.



Epoch 1: LearningRateScheduler setting learning rate to 0.0001.
Epoch 1/150


2025-07-29 14:49:17.978893: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


In [ ]:
model.save('transformer_train_model.keras')
model.save('transformer_val_model.keras')

In [ ]:
from google.colab import files

# Download the file
files.download('./transformer_train_model.keras')
files.download('./transformer_val_model.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model = load_model("transformer_val_model.keras", custom_objects={"custom_mae_loss": custom_mae_loss, "dir_acc": dir_acc, "get_lr_callback": get_lr_callback})

accuracy = model.evaluate(test_sequences, test_labels)[1]  # Evaluate the model on the test data
print(accuracy)

predictions = model.predict(test_sequences)  # Make predictions on the test dataset
pred_rescaled = predictions[:, 0] * stats['dataClose'][1] + stats['dataClose'][0]
actual_rescaled = test_labels[:, 1] * stats['dataClose'][1] + stats['dataClose'][0]
r2 = r2_score(actual_rescaled, pred_rescaled)
print("R2 Score:", r2)


642/642 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - dir_acc: 0.6674 - loss: 0.3071
0.6663606762886047
642/642 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step
R2 Score: 0.1006097105606304
